# Data loading

In [1]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

cifar10_classes = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

cifar10_class_to_idx = {v: k for k, v in cifar10_classes.items()}

label_transform = lambda label: cifar10_classes[label]

dataset = torchvision.datasets.CIFAR10(
    root='/home/azureuser/gautijha37/vision/data',      # Directory where the data will be stored/loaded from
    train=True,        # Load the training set
    download=True,     # Download the dataset if it's not already present
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5),
                         std=(0.5, 0.5, 0.5))
    ]),
    target_transform=label_transform   
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Use random_split to safely create subsets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from transformers import SiglipModel, SiglipImageProcessor, SiglipTokenizer, SiglipConfig, SiglipVisionConfig, SiglipTextConfig, SiglipVisionModel, SiglipTextModel

config = SiglipConfig()
config.vision_config.image_size=32
config.text_config.max_position_embeddings=4 # max length of tokenized classes

model = SiglipModel(config).to(device)

img_processor = SiglipImageProcessor(do_resize=False, do_rescale=False) # transforms.ToTensor() already scales input img to [-1, 1]
tokenizer = SiglipTokenizer.from_pretrained("google/siglip-base-patch16-224")


/home/azureuser/gautijha37/vision/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [12]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb

num_epochs = 20
# base_lr = 1e-3
# ref_batch_size = 4096
batch_size = 64
# lr = base_lr * torch.sqrt(batch_size/ref_batch_size)
lr = 1e-4

wandb.init(project="siglip-cifar10", config={
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": lr,
    "model": "SigLIP",
    "dataset": "CIFAR10"
})

def collate_fn(batch):
    images, labels = zip(*batch)
    img_inputs = img_processor(images=images, return_tensors="pt")
    text_inputs = tokenizer(text = labels, padding=True, return_tensors="pt")
    
    labels_ids = torch.tensor([cifar10_class_to_idx[label] for label in labels])
    batch_mask = (labels_ids.unsqueeze(1) == labels_ids.unsqueeze(0)).float()
    batch_mask = 2 * batch_mask - 1
    
    return img_inputs, text_inputs, batch_mask

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16, collate_fn=collate_fn)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)

for epoch in tqdm(range(num_epochs)):
    model.train()
    for batch_id, (img_inputs, text_inputs, batch_mask) in enumerate(train_loader):
        t0 = time.time()
        
        img_inputs['pixel_values']=img_inputs['pixel_values'].to(device)
        text_inputs['input_ids']=text_inputs['input_ids'].to(device)
        batch_mask=batch_mask.to(device)
        
        # loss calculation
        output = model(**text_inputs, **img_inputs)
        loglik = torch.nn.functional.logsigmoid(batch_mask * output.logits_per_text)
        nll = -torch.sum(loglik, dim=-1)
        loss = nll.mean()
        
        optimizer.zero_grad()
        loss.backward()
        
        wandb.log({
            "batch_loss": loss.item(),
            "epoch": epoch,
            "batch": batch_id,
            "step": epoch * len(train_loader) + batch_id
        })
        if (batch_id + 1) % 125 == 0:
            print(f"batched loss: {epoch:}, {batch_id + 1:}, {loss:}")
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")
        

  0%|          | 0/20 [00:00<?, ?it/s]

batched loss: 0, 125, 8.251632690429688
batched loss: 0, 250, 10.437765121459961
batched loss: 0, 375, 7.887005805969238
batched loss: 0, 500, 8.607136726379395


  5%|▌         | 1/20 [00:32<10:18, 32.55s/it]

batched loss: 0, 625, 11.275980949401855
Epoch [1/20], Loss: 11.2760
batched loss: 1, 125, 12.086872100830078
batched loss: 1, 250, 10.13748550415039
batched loss: 1, 375, 10.029108047485352
batched loss: 1, 500, 9.858497619628906


 10%|█         | 2/20 [01:04<09:39, 32.21s/it]

...

batched loss: 17, 625, 1.461958408355713
Epoch [18/20], Loss: 1.4620
batched loss: 18, 125, 0.5359780192375183
batched loss: 18, 250, 0.5427296757698059
batched loss: 18, 375, 0.5220982432365417
batched loss: 18, 500, 0.5435538291931152


 95%|█████████▌| 19/20 [10:14<00:32, 32.33s/it]

batched loss: 18, 625, 0.5519874095916748
Epoch [19/20], Loss: 0.5520
batched loss: 19, 125, 0.5661091804504395
batched loss: 19, 250, 2.934039354324341
batched loss: 19, 375, 1.2906641960144043
batched loss: 19, 500, 0.5394741296768188


100%|██████████| 20/20 [10:46<00:00, 32.31s/it]

batched loss: 19, 625, 0.5377059578895569
Epoch [20/20], Loss: 0.5377


# Validation

In [13]:
from safetensors.torch import save_file
state_dict = model.state_dict()
save_file(state_dict, "/home/azureuser/gautijha37/vision/siglip.safetensors5")

In [14]:
model.eval()

SiglipModel(
  (text_model): SiglipTextTransformer(
    (embeddings): SiglipTextEmbeddings(
      (token_embedding): Embedding(32000, 768)
      (position_embedding): Embedding(4, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (self_attn): SiglipSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,

In [15]:
from torch.utils.data import DataLoader
from tqdm import tqdm

val_loader = DataLoader(val_dataset, batch_size=len(cifar10_classes), num_workers=16, shuffle=True)
correct = 0

all_text_inputs = list(cifar10_classes.values())
text_inputs = tokenizer(text=all_text_inputs, padding=True, return_tensors="pt")
text_inputs['input_ids']=text_inputs['input_ids'].to(device)

for images, correct_labels in tqdm(val_loader):
    img_inputs = img_processor(images=images, return_tensors="pt")
    img_inputs['pixel_values']=img_inputs['pixel_values'].to(device)
    
    output = model(**text_inputs, **img_inputs)
    predicted_indices = output.logits_per_image.argmax(dim=1)
    predicted_labels = [cifar10_classes[k.item()] for k in predicted_indices]
    
    correct += sum(p == c for p, c in zip(predicted_labels, list(correct_labels)))

total = len(val_loader) * val_loader.batch_size
100 * correct/total

100%|██████████| 1000/1000 [00:15<00:00, 66.49it/s]


45.33

# Testing

In [16]:
test_dataset = torchvision.datasets.CIFAR10(
    root='/home/azureuser/gautijha37/vision/data',
    train=False,        # Load the test set
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5),
                         std=(0.5, 0.5, 0.5))
    ]),
    target_transform=label_transform
)

# Testing on test dataset
test_loader = DataLoader(test_dataset, batch_size=len(cifar10_classes), num_workers=16, shuffle=True)
correct = 0

all_text_inputs = list(cifar10_classes.values())
text_inputs = tokenizer(text=all_text_inputs, padding=True, return_tensors="pt")
text_inputs['input_ids']=text_inputs['input_ids'].to(device)

for images, correct_labels in tqdm(test_loader):
    img_inputs = img_processor(images=images, return_tensors="pt")
    img_inputs['pixel_values']=img_inputs['pixel_values'].to(device)
    
    output = model(**text_inputs, **img_inputs)
    predicted_indices = output.logits_per_image.argmax(dim=1)
    predicted_labels = [cifar10_classes[k.item()] for k in predicted_indices]
    
    correct += sum(p == c for p, c in zip(predicted_labels, list(correct_labels)))

total = len(test_loader) * test_loader.batch_size
test_accuracy = 100 * correct/total
print(f"Test Accuracy: {test_accuracy:.2f}%")

100%|██████████| 1000/1000 [00:15<00:00, 65.54it/s]

Test Accuracy: 46.87%
